In [3]:
# this notebook will use a basic GPT based decision transformer in offline reinforcement learning setting to create bot for trading stock

In [ ]:
# import libraries
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# based on https://github.com/nikhilbarhate99/min-decision-transformer/blob/master/decision_transformer/model.py

# define the masked causal attention
class MaskedAttention(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()
        self.n_heads = n_heads
        self.drop_p = drop_p
        # feed forward networks which create the query, key and value
        self.Q_net = nn.Linear(h_dim, h_dim)
        self.K_net = nn.Linear(h_dim, h_dim)
        self.V_net = nn.Linear(h_dim, h_dim)

        # feed forward network which projects the attention to the correct dimension
        self.proj_net = nn.Linear(h_dim, h_dim)

        # dropout layers
        self.att_drop = nn.Dropout(drop_p)
        self.proj_drop = nn.Dropout(drop_p)

        # create the mask
        mask = torch.tril(torch.ones(max_T, max_T)).view(1, 1, max_T, max_T)

        # register_buffer will make the mask a constant tensor
        # so that it will not be included in the model parameters and be updated during backpropagation
        self.register_buffer('mask', mask)

    def forward(self, x):
        # x: [B, T, H]
        B, T, C = x.shape # batch size, sequence length, hidden dimension * number of heads
        N, D = self.n_heads, C // self.n_heads # number of heads, dimension of each head

        # compute the query, key and value
        Q = self.Q_net(x).view(B, T, N, D).transpose(1, 2) # [B, N, T, D]
        K = self.K_net(x).view(B, T, N, D).transpose(1, 2)
        V = self.V_net(x).view(B, T, N, D).transpose(1, 2)

        # compute the attention
        weights = Q @ K.transpose(2,3) / math.sqrt(D) # QK^T / sqrt(D)
        weights = weights.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf')) # mask the future tokens
        normalized_weights = F.softmax(weights, dim=-1) # softmax along the last dimension
        A = self.att_drop(normalized_weights) # attention with dropout

        # compute the output
        # gather heads and project to correct dimension
        attention = attention.transpose(1, 2).contiguous().view(B, T, N*D)
        out = self.proj_drop(self.proj_net(attention))

        return out

# define the attention block with layer normalization and residual connection as well as the feed forward network
class AttentionBlock(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()
        self.attention = MaskedAttention(h_dim, max_T, n_heads, drop_p)
        self.norm1 = nn.LayerNorm(h_dim)
        self.norm2 = nn.LayerNorm(h_dim)
        self.ffn = nn.Sequential(
            nn.Linear(h_dim, 4*h_dim),
            nn.GELU(),
            nn.Linear(4*h_dim, h_dim),
            nn.Dropout(drop_p)
        )

    def forward(self, x):
        # x: [B, T, H]
        # Attention -> LayerNorm -> Residual -> FFN -> LayerNorm -> Residual
        out = self.norm1(x + self.attention(x))
        out = self.norm2(out + self.ffn(out))

        return out

# define the decision transformer
class DecisionTransformer(nn.Module):
    def __init__(self, state_dim, act_dim, n_block, h_dim, context_len, n_heads, drop_p, max_timestep = 4096):
        super().__init__()
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.h_dim = h_dim

        # transformer blocks
        input_seq_len = 3 * context_len
        blocks = [AttentionBlock(h_dim, input_seq_len, n_heads, drop_p) for _ in range(n_block)]
        self.transformer = nn.Sequential(*blocks)

        # projection heads (project to embedding)
        self.embed_ln = nn.LayerNorm(h_dim)
        self.embed_timestep = nn.Embedding(max_timestep, h_dim)
        self.embed_rtg = nn.Linear(1, h_dim)
        self.embed_state = nn.Linear(state_dim, h_dim)

        # discrete actions
        self.embed_act = nn.Embedding(act_dim, h_dim)
        use_action_tah = False # for discrete action

        # prediction heads
        self.pred_rtg = nn.Linear(h_dim, 1)
        self.pred_state = nn.Linear(h_dim, state_dim)
        self.pred_act = nn.Sequential(*([nn.Linear(h_dim, act_dim)] + ([nn.Tanh()] if use_action_tah else [])))
    
    def forward(self, state, rtg, timestep, actions):
        B, T, _ = state.shape

        # timestep embedding
        time_emb = self.embed_timestep(timestep)

        # embedding for the state, reward and actions along with time embedding
        state_emb = self.embed_state(state) + time_emb
        rtg_emb = self.embed_rtg(rtg) + time_emb
        act_emb = self.embed_act(actions) + time_emb

        # stack the embeddings and reshape sequence as (r1, s1, a1, r2, s2, a2, ...)
        h = torch.stack([rtg_emb, state_emb, act_emb], dim=1).permute(0,2,1,3).reshape(B, 3*T, self.h_dim)
        h = self.embed_ln(h)

        # transformer blocks
        h = self.transformer(h)

        # get h reshaped such that its size is (B, 3, T, h_dim) and
        # h[:, 0, t] is conditioned on r_0, s_0, a_0, ..., r_t
        # h[:, 1, t] is conditioned on r_0, s_0, a_0, ..., r_t, s_t
        # h[:, 2, t] is conditioned on r_0, s_0, a_0, ..., r_t, s_t, a_t
        h = h.reshape(B, 3, T, self.h_dim).permute(0,2,1,3)

        # get predictions
        return_preds = self.pred_rtg(h[:,2])    # predict next rtg given r, s, a
        state_preds = self.pred_state(h[:,2])   # predict next state given r, s, a
        act_preds = self.pred_act(h[:,2])       # predict action given r, s

        return return_preds, state_preds, act_preds

In [1]:
# load huggingface dataset from json file
filename = 'AAPL_replaybuffer.json'

from datasets import load_dataset
dataset = load_dataset("json", data_files=filename, field='data').with_format('torch')

/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-ba70c097613802b6
Found cached dataset json (/home/victoru/.cache/huggingface/datasets/json/default-ba70c097613802b6/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 988.52it/s]


In [3]:
dataset['train'][0]['timestep'][0:60]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27,  0,  1,  2,  3,  4,  5,  6,  7,
         8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
        26, 27,  0,  1,  2,  3])

In [3]:
# create dataloader from dataset
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset['train'], batch_size=2)

In [10]:
for batch in dataloader:
    print(batch['state'])
    break


tensor([[[ 3.8557e+01,  1.4816e+08,  2.8817e+08,  3.5896e-03,  1.0000e+02],
         [ 3.5500e+01,  3.6525e+08, -7.7076e+07, -3.0734e-01,  1.0514e+00],
         [ 3.5950e+01,  2.3443e+08,  1.5735e+08, -4.2639e-01,  2.9883e+01],
         ...,
         [ 1.7707e+02,  7.4920e+07,  5.1828e+09,  5.0512e+00,  6.6890e+01],
         [ 1.7853e+02,  7.9144e+07,  5.1037e+09,  5.1029e+00,  6.4950e+01],
         [ 1.7814e+02,  6.2349e+07,  5.1660e+09,  5.0925e+00,  6.5045e+01]],

        [[ 3.8557e+01,  1.4816e+08,  2.8817e+08,  3.5896e-03,  1.0000e+02],
         [ 3.5500e+01,  3.6525e+08, -7.7076e+07, -3.0734e-01,  1.0514e+00],
         [ 3.5950e+01,  2.3443e+08,  1.5735e+08, -4.2639e-01,  2.9883e+01],
         ...,
         [ 1.7707e+02,  7.4920e+07,  5.1828e+09,  5.0512e+00,  6.6890e+01],
         [ 1.7853e+02,  7.9144e+07,  5.1037e+09,  5.1029e+00,  6.4950e+01],
         [ 1.7814e+02,  6.2349e+07,  5.1660e+09,  5.0925e+00,  6.5045e+01]]])
